# 数据小白入门教程

数据小白入门教程主要通过阿里天池平台的新手入门赛中的[阿里移动推荐算法](https://tianchi.shuju.aliyun.com/getStart/introduction.htm?spm=5176.100066.333.3.iY9ga2&raceId=231522)为基本案例，来阐述怎样使用数据取解决问题或者参加比赛，若大家进一步对数据分析或挖掘比较感兴趣，可以参看阿里天池[新手入门视频](https://tianchi.shuju.aliyun.com/video.htm?spm=5176.100067.1234.7.3hqMDK)学习，同时可以通过[比赛](https://tianchi.shuju.aliyun.com/competition/index.htm?spm=5176.7953375.1234.2.tqpeV8&id=)去强化技术水平。




## 赛题解读

### 1. 干啥的

根据用户在手机淘宝上的历史行为数据，预测用户们第二天买了什么

1. 历史行为记录是什么？

谁在什么时间对某一物品干啥了。

2. 到底预测的是什么？

在第32天中，谁买了什么。

### 2. 咋评价

如果比预测的多：那么提交所有种可能，必得满分

如果比预测的准：只预测一条对的，必得满分

因此评价指标必须要兼顾到“多”和“准”

--------
例子

假设第31天(12月19号)发生了20000次交易，我们预测了10000条，预测对了1000条

准确率： 1000/10000 = 0.1
召回率： 1000/20000 = 0.05

F1 = 2 * 0.1 * 0.05 / (0.1 + 0.05)


### 3. 咋做

我们先从业务常识去判断，什么样的user-item pair，可能在下一天中发生"购买"：

1. 用户i不停的去看商品j, 预测: $user_i-item_j$

2. 用户i将商品j放入购物车，预测: $user_i-item_j$

3. 用户i非常喜欢买东西，预测和$user_i$相关的所有pair

4. 商品j这几天卖得很好，预测和$item_j$相关的所有pair


### 4. 咋做好

如何将多个规则结合在一起：

1. 按照每一个规则是否达标进行打分，打分最高的当作预测

2. 自动的确定分数: Logistics Regression

3. 如果诸多规则和目标之间的关系是非线性的咋办： More Machine Learning Algorithm



## 数据处理

### 1. 移动端行为数据加载到MySQL

#### 1.1建表

```sql

CREATE TABLE IF NOT EXISTS mobile_recomm_user_log_2w (
    `user_id` varchar(255) DEFAULT NULL,
    `item_id` varchar(255) DEFAULT NULL,
    `behavior_type` int(4) DEFAULT 0,
    `user_geohash` varchar(255) DEFAULT NULL,
    `date` DATE,
    `time` int(4)
) ENGINE=MyISAM DEFAULT CHARSET=utf8;

```

In [1]:
import sys
datamaker_root = '../'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, datamaker_root + 'tutorial')

import os

print os.getcwd()

from mysql_helper import MySQLHelper
from config_helper import Config_Read

creat_table_sql = """
    CREATE TABLE IF NOT EXISTS mobile_recomm_user_log_2w (
    `user_id` varchar(255) DEFAULT NULL,
    `item_id` varchar(255) DEFAULT NULL,
    `behavior_type` int(4) DEFAULT 0,
    `user_geohash` varchar(255) DEFAULT NULL,
    `dt` DATE,
    `time` int(4)
) ENGINE=MyISAM DEFAULT CHARSET=utf8;
"""
# 获取MySQL配置信息
mysql_info = Config_Read('mysql')._get_config_info
print mysql_info

# 连接MySQL 
mysqler = MySQLHelper(mysql_info)

/Users/EricDoug/Documents/myDevelopment/dataMaker/tutorial
{'passwd': '123456', 'host': 'localhost', 'db': 'tutorial', 'port': 3306, 'user': 'root'}


In [2]:
# 创建表
mysqler.execute(creat_table_sql)

True

#### 1.2 数据加载到MySQL

In [3]:
import datetime

data_dir = "/Users/EricDoug/Documents/datas/ali/fresh_comp_offline"
file_name = "tianchi_fresh_comp_train_user.csv"
train_data = os.path.join(data_dir,file_name)


# 按行加载数据到MySQL

with open(train_data) as f:
    # read header
    header = f.readline()
    print header
    rows = f.readlines()
    # 统计加载数据时间
    start_time = datetime.datetime.now()
    for row in rows:
        # 去空白
        row = row.strip()
        # 使用“,”wei提取
        cols = row.split(',')
        user_id = cols[0]
        item_id = cols[1]
        behavior_type = cols[2]
        user_geohash = cols[3]
        item_category = cols[4]
        dtime= cols[5]
        date_time = dtime.split(' ')
        dt = date_time[0]
        time = date_time[1]
        #print user_id, item_id, dt, time
        insert_sql_str = "INSERT INTO mobile_recomm_user_log_2w(user_id, item_id, behavior_type, user_geohash, dt, time)  \
                 VALUES ('%s', '%s', %s, '%s', '%s' , %s)"
        
        insert_sql = insert_sql_str % (user_id, item_id, behavior_type, user_geohash, dt, time)
        #print insert_sql
        # 按行插入2万行
        mysqler.execute(insert_sql)
        
    end_time = datetime.datetime.now()
    print "load data spend:" (end_time - start_time).seconds 
        

        

user_id,item_id,behavior_type,user_geohash,item_category,time



## 特征提取

赛题给的是2014年11月18日到2014年12月18日，需要预测2014年12月19日这一天用户的购买。为了有效对模型进行评估，我们必须划分一个离线的模型校验数据集，使用12月18日的数据为测试数据集，将11月18日到12月17日的数据用于模型训练。由于一个月时间的用户行为大致为2000万条记录，使用所有数据计算量有点大，为了减少计算量，我们采用预测前一周的用户行为进行模型训练，下面主要使用一周的数据构建特征(若机器性能允许，可以考虑使用一个月数据)。

### 1. 该用户是否会周期性购买某一物品

1.前1天(12月17日)是否购买，购买为1， 没有购买为0
2.前2天(12月16日)是否购买，购买为1， 没有购买为0
3.前3天(12月15日)是否购买，购买为1， 没有购买为0
4.前4天(12月14日)是否购买，购买为1， 没有购买为0
5.前5天(12月13日)是否购买，购买为1， 没有购买为0
6.前6天(12月12日)是否购买，购买为1， 没有购买为0
7.前7天(12月11日)是否购买，购买为1， 没有购买为0

```sql
    SELECT ui.user_id, ui.item_id,
    IF(buy_1.user_id = NULL,0,1) 
    FROM 
    (SELECT user_id, item_id FROM mobile_recomm_user_log_2w WHERE dt BETWEEN '2014-12-11'
    AND '2014-12-17' group by user_id,item_id) AS ui
    LEFT JOIN
    (SELECT user_id, item_id FROM mobile_recomm_user_log_2w WHERE dt = '2014-12-17') buy_1
    ON ui.user_id = buy_1.user_id and ui.item_id = buy_1.item_id
    

```

### 2. 加入购物车与购买的关系

8前1天内(12月17)加入购物车的次数
9.前2天内(12.16~12.17)加入购物车的次数
10.前3天内(12.15~12.17)加入购物车的次数
11.前5天内(12.13~12.17)加入购物车的次数
12.前7天内(12.11~12.17)加入购物车的次数

### 3. 收藏与购买的关系
13.前1天内(12月17)加入收藏的次数
14.前2天内(12.16~12.17)加入收藏的次数
15.前3天内(12.15~12.17)加入收藏的次数
16.前5天内(12.13~12.17)加入收藏的次数
17.前7天内(12.11~12.17)加入收藏的次数

### 4. 浏览与购买的关系

18.前1天内(12月17)加入浏览的次数
19.前2天内(12.16~12.17)加入浏览的次数
20.前3天内(12.15~12.17)加入浏览的次数
21.前5天内(12.13~12.17)加入浏览的次数
22.前7天内(12.11~12.17)加入浏览的次数



## 构建模型

### 1. Logister Regression


##### 1.1 Sklearn实现

#### 1.2 Tensorflow 实现

## 模型验证

## 预测